In [1]:
import os
import re
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_colwidth = 160

from nltk import sent_tokenize

import features as util
from raw_utils import save_to_csv
from preprocessing import tokenize

### Read Data

In [2]:
# Path
cwd = os.getcwd()
csv_path = os.path.join(cwd, 'data/csv/')

train_text = ['train_balanced_text.csv', 'train_imbalanced_text.csv']
test_text = ['test_balanced_text.csv', 'test_imbalanced_text.csv']

#### Email Text

In [3]:
train_balanced_text = pd.read_csv(os.path.join(csv_path, train_text[0]), index_col=0)
test_balanced_text = pd.read_csv(os.path.join(csv_path, test_text[0]), index_col=0)

In [4]:
train_imbalanced_text = pd.read_csv(os.path.join(csv_path, train_text[1]), index_col=0)
test_imbalanced_text = pd.read_csv(os.path.join(csv_path, test_text[1]), index_col=0)

After the preprocessing, the data look like this:

In [5]:
test_balanced_text.head()

,id,body,class
0,2377,please print\n----- Forwarded by Steven J Kean/NA/Enron on 10/16/2000 10:27 AM -----\n\n\tCynthia Sandherr\n\t10/12/2000 07:43 PM\n\t\t \n\t\t To: Thomas E ...,False
1,592,Server Message \n \n\n \n Dear <emailaddress> Our record indicates that you recently made a request to deactivate email And this request will be process...,True
2,1049,Please see the attached articles:,False
3,1087,Please Click Here<urladdress> to Update e-mail Password\n\n\n\nIT Security immediately/\n\n________________________________\nSEED IS PROUD TO BE A 21st CENT...,True
4,725,"Wednesday afternoon Febuary 6th, 2002, the Enron building experienced a brief power outage. The building is powered by one of two Reliant circuits. Yester...",False


#### Email Tokens

Since the .csv files with the already tokenized emails have been subject to further preprocessing like lemmatization and stopword removal, a simple tokenization (at word and sentence level) will also be run here for the purposes of feature extraction.

In [6]:
train_balanced_text['tokens'] = train_balanced_text['body'].apply(tokenize)
test_balanced_text['tokens'] = test_balanced_text['body'].apply(tokenize)
train_imbalanced_text['tokens'] = train_imbalanced_text['body'].apply(tokenize)
test_imbalanced_text['tokens'] = test_imbalanced_text['body'].apply(tokenize)

Note that for the sentence-level tokenization, `nltk.sent_tokenization` is used, so any sentences separated by a newline without punctuation will be considered simply as wrapped text, and not new, different sentences.

In [7]:
train_balanced_text['sentences'] = train_balanced_text['body'].apply(sent_tokenize)
test_balanced_text['sentences'] = test_balanced_text['body'].apply(sent_tokenize)
train_imbalanced_text['sentences'] = train_imbalanced_text['body'].apply(sent_tokenize)
test_imbalanced_text['sentences'] = test_imbalanced_text['body'].apply(sent_tokenize)

# Stylometric Features

A useful marker of wether an email is phishing or not should stem from the writing style of the author.<br>
With this in mind, several features that have been used in the literature will be extracted, in order to be compared and combined with the text-only baseline.

In [8]:
train_balanced_style = train_balanced_text[['id', 'class']].copy()
test_balanced_style = test_balanced_text[['id', 'class']].copy()
train_imbalanced_style = train_imbalanced_text[['id', 'class']].copy()
test_imbalanced_style = test_imbalanced_text[['id', 'class']].copy()

### Simple Counts

The simplest kind of features would be simple counts of various parts of the emails, like characters and words.

In [9]:
train_balanced_style['num_chars'] = train_balanced_text['body'].apply(util.count_chars)
train_balanced_style['num_newlines'] = train_balanced_text['body'].apply(util.count_newlines)
train_balanced_style['num_words'] = train_balanced_text['tokens'].apply(util.count_words)
train_balanced_style['num_unique_words'] = train_balanced_text['tokens'].apply(util.count_unique_words)
train_balanced_style['sentences'] = train_balanced_text['sentences'].apply(util.count_sentences)
train_balanced_style[['num_sentences', 'num_upper_sentences', 'num_lower_sentences']] = pd.DataFrame(train_balanced_style['sentences'].tolist(), index=train_balanced_style.index)
train_balanced_style = train_balanced_style.drop('sentences', axis=1)

In [10]:
test_balanced_style['num_chars'] = test_balanced_text['body'].apply(util.count_chars)
test_balanced_style['num_newlines'] = test_balanced_text['body'].apply(util.count_newlines)
test_balanced_style['num_words'] = test_balanced_text['tokens'].apply(util.count_words)
test_balanced_style['num_unique_words'] = test_balanced_text['tokens'].apply(util.count_unique_words)
test_balanced_style['sentences'] = test_balanced_text['sentences'].apply(util.count_sentences)
test_balanced_style[['num_sentences', 'num_upper_sentences', 'num_lower_sentences']] = pd.DataFrame(test_balanced_style['sentences'].tolist(), index=test_balanced_style.index)
test_balanced_style = test_balanced_style.drop('sentences', axis=1)

In [11]:
train_imbalanced_style['num_chars'] = train_imbalanced_text['body'].apply(util.count_chars)
train_imbalanced_style['num_newlines'] = train_imbalanced_text['body'].apply(util.count_newlines)
train_imbalanced_style['num_words'] = train_imbalanced_text['tokens'].apply(util.count_words)
train_imbalanced_style['num_unique_words'] = train_imbalanced_text['tokens'].apply(util.count_unique_words)
train_imbalanced_style['sentences'] = train_imbalanced_text['sentences'].apply(util.count_sentences)
train_imbalanced_style[['num_sentences', 'num_upper_sentences', 'num_lower_sentences']] = pd.DataFrame(train_imbalanced_style['sentences'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style = train_imbalanced_style.drop('sentences', axis=1)

In [12]:
test_imbalanced_style['num_chars'] = test_imbalanced_text['body'].apply(util.count_chars)
test_imbalanced_style['num_newlines'] = test_imbalanced_text['body'].apply(util.count_newlines)
test_imbalanced_style['num_words'] = test_imbalanced_text['tokens'].apply(util.count_words)
test_imbalanced_style['num_unique_words'] = test_imbalanced_text['tokens'].apply(util.count_unique_words)
test_imbalanced_style['sentences'] = test_imbalanced_text['sentences'].apply(util.count_sentences)
test_imbalanced_style[['num_sentences', 'num_upper_sentences', 'num_lower_sentences']] = pd.DataFrame(test_imbalanced_style['sentences'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style = test_imbalanced_style.drop('sentences', axis=1)

## Word Size

Another category of features are those that related to the size of words, like the average size and counts or frequencies of smaller or bigger words.

In [13]:
train_balanced_style['avg_word_size'] = train_balanced_text['tokens'].apply(util.average_word_length)
train_balanced_style['small_words'] = train_balanced_text['tokens'].apply(util.small_words)
train_balanced_style[['num_small_words', 'freq_small_words']] = pd.DataFrame(train_balanced_style['small_words'].tolist(), index=train_balanced_style.index)
train_balanced_style['big_words'] = train_balanced_text['tokens'].apply(util.big_words)
train_balanced_style[['num_big_words', 'freq_big_words']] = pd.DataFrame(train_balanced_style['big_words'].tolist(), index=train_balanced_style.index)
train_balanced_style['huge_words'] = train_balanced_text['tokens'].apply(util.huge_words)
train_balanced_style[['num_huge_words', 'freq_huge_words']] = pd.DataFrame(train_balanced_style['huge_words'].tolist(), index=train_balanced_style.index)
train_balanced_style = train_balanced_style.drop(['small_words', 'big_words', 'huge_words'], axis=1)

In [14]:
test_balanced_style['avg_word_size'] = test_balanced_text['tokens'].apply(util.average_word_length)
test_balanced_style['small_words'] = test_balanced_text['tokens'].apply(util.small_words)
test_balanced_style[['num_small_words', 'freq_small_words']] = pd.DataFrame(test_balanced_style['small_words'].tolist(), index=test_balanced_style.index)
test_balanced_style['big_words'] = test_balanced_text['tokens'].apply(util.big_words)
test_balanced_style[['num_big_words', 'freq_big_words']] = pd.DataFrame(test_balanced_style['big_words'].tolist(), index=test_balanced_style.index)
test_balanced_style['huge_words'] = test_balanced_text['tokens'].apply(util.huge_words)
test_balanced_style[['num_huge_words', 'freq_huge_words']] = pd.DataFrame(test_balanced_style['huge_words'].tolist(), index=test_balanced_style.index)
test_balanced_style = test_balanced_style.drop(['small_words', 'big_words', 'huge_words'], axis=1)

In [15]:
train_imbalanced_style['avg_word_size'] = train_imbalanced_text['tokens'].apply(util.average_word_length)
train_imbalanced_style['small_words'] = train_imbalanced_text['tokens'].apply(util.small_words)
train_imbalanced_style[['num_small_words', 'freq_small_words']] = pd.DataFrame(train_imbalanced_style['small_words'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style['big_words'] = train_imbalanced_text['tokens'].apply(util.big_words)
train_imbalanced_style[['num_big_words', 'freq_big_words']] = pd.DataFrame(train_imbalanced_style['big_words'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style['huge_words'] = train_imbalanced_text['tokens'].apply(util.huge_words)
train_imbalanced_style[['num_huge_words', 'freq_huge_words']] = pd.DataFrame(train_imbalanced_style['huge_words'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style = train_imbalanced_style.drop(['small_words', 'big_words', 'huge_words'], axis=1)

In [16]:
test_imbalanced_style['avg_word_size'] = test_imbalanced_text['tokens'].apply(util.average_word_length)
test_imbalanced_style['small_words'] = test_imbalanced_text['tokens'].apply(util.small_words)
test_imbalanced_style[['num_small_words', 'freq_small_words']] = pd.DataFrame(test_imbalanced_style['small_words'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style['big_words'] = test_imbalanced_text['tokens'].apply(util.big_words)
test_imbalanced_style[['num_big_words', 'freq_big_words']] = pd.DataFrame(test_imbalanced_style['big_words'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style['huge_words'] = test_imbalanced_text['tokens'].apply(util.huge_words)
test_imbalanced_style[['num_huge_words', 'freq_huge_words']] = pd.DataFrame(test_imbalanced_style['huge_words'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style = test_imbalanced_style.drop(['small_words', 'big_words', 'huge_words'], axis=1)

## Sentence Size

Another set of features could be related to various simple statistics about the size of a sentence, using both characters and words as a unit of size.

In [17]:
train_balanced_style['avg_sent_size'] = train_balanced_text['sentences'].apply(util.average_sentence_length)
train_balanced_style[['avg_sentence_chars', 'avg_sentence_words']] = pd.DataFrame(train_balanced_style['avg_sent_size'].tolist(), index=train_balanced_style.index)
train_balanced_style['std_sent_size'] = train_balanced_text['sentences'].apply(util.std_sentence_length)
train_balanced_style[['std_sentence_chars', 'std_sentence_words']] = pd.DataFrame(train_balanced_style['std_sent_size'].tolist(), index=train_balanced_style.index)
train_balanced_style['min_sent_size'] = train_balanced_text['sentences'].apply(util.min_sentence_length)
train_balanced_style[['min_sentence_chars', 'min_sentence_words']] = pd.DataFrame(train_balanced_style['min_sent_size'].tolist(), index=train_balanced_style.index)
train_balanced_style['max_sent_size'] = train_balanced_text['sentences'].apply(util.max_sentence_length)
train_balanced_style[['max_sentence_chars', 'max_sentence_words']] = pd.DataFrame(train_balanced_style['max_sent_size'].tolist(), index=train_balanced_style.index)
train_balanced_style = train_balanced_style.drop(['avg_sent_size', 'std_sent_size', 'min_sent_size', 'max_sent_size'], axis=1)

In [18]:
test_balanced_style['avg_sent_size'] = test_balanced_text['sentences'].apply(util.average_sentence_length)
test_balanced_style[['avg_sentence_chars', 'avg_sentence_words']] = pd.DataFrame(test_balanced_style['avg_sent_size'].tolist(), index=test_balanced_style.index)
test_balanced_style['std_sent_size'] = test_balanced_text['sentences'].apply(util.std_sentence_length)
test_balanced_style[['std_sentence_chars', 'std_sentence_words']] = pd.DataFrame(test_balanced_style['std_sent_size'].tolist(), index=test_balanced_style.index)
test_balanced_style['min_sent_size'] = test_balanced_text['sentences'].apply(util.min_sentence_length)
test_balanced_style[['min_sentence_chars', 'min_sentence_words']] = pd.DataFrame(test_balanced_style['min_sent_size'].tolist(), index=test_balanced_style.index)
test_balanced_style['max_sent_size'] = test_balanced_text['sentences'].apply(util.max_sentence_length)
test_balanced_style[['max_sentence_chars', 'max_sentence_words']] = pd.DataFrame(test_balanced_style['max_sent_size'].tolist(), index=test_balanced_style.index)
test_balanced_style = test_balanced_style.drop(['avg_sent_size', 'std_sent_size', 'min_sent_size', 'max_sent_size'], axis=1)

In [19]:
train_imbalanced_style['avg_sent_size'] = train_imbalanced_text['sentences'].apply(util.average_sentence_length)
train_imbalanced_style[['avg_sentence_chars', 'avg_sentence_words']] = pd.DataFrame(train_imbalanced_style['avg_sent_size'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style['std_sent_size'] = train_imbalanced_text['sentences'].apply(util.std_sentence_length)
train_imbalanced_style[['std_sentence_chars', 'std_sentence_words']] = pd.DataFrame(train_imbalanced_style['std_sent_size'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style['min_sent_size'] = train_imbalanced_text['sentences'].apply(util.min_sentence_length)
train_imbalanced_style[['min_sentence_chars', 'min_sentence_words']] = pd.DataFrame(train_imbalanced_style['min_sent_size'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style['max_sent_size'] = train_imbalanced_text['sentences'].apply(util.max_sentence_length)
train_imbalanced_style[['max_sentence_chars', 'max_sentence_words']] = pd.DataFrame(train_imbalanced_style['max_sent_size'].tolist(), index=train_imbalanced_style.index)
train_imbalanced_style = train_imbalanced_style.drop(['avg_sent_size', 'std_sent_size', 'min_sent_size', 'max_sent_size'], axis=1)

In [20]:
test_imbalanced_style['avg_sent_size'] = test_imbalanced_text['sentences'].apply(util.average_sentence_length)
test_imbalanced_style[['avg_sentence_chars', 'avg_sentence_words']] = pd.DataFrame(test_imbalanced_style['avg_sent_size'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style['std_sent_size'] = test_imbalanced_text['sentences'].apply(util.std_sentence_length)
test_imbalanced_style[['std_sentence_chars', 'std_sentence_words']] = pd.DataFrame(test_imbalanced_style['std_sent_size'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style['min_sent_size'] = test_imbalanced_text['sentences'].apply(util.min_sentence_length)
test_imbalanced_style[['min_sentence_chars', 'min_sentence_words']] = pd.DataFrame(test_imbalanced_style['min_sent_size'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style['max_sent_size'] = test_imbalanced_text['sentences'].apply(util.max_sentence_length)
test_imbalanced_style[['max_sentence_chars', 'max_sentence_words']] = pd.DataFrame(test_imbalanced_style['max_sent_size'].tolist(), index=test_imbalanced_style.index)
test_imbalanced_style = test_imbalanced_style.drop(['avg_sent_size', 'std_sent_size', 'min_sent_size', 'max_sent_size'], axis=1)

The calculated features look like this:

In [21]:
test_balanced_style.head()

,id,class,num_chars,num_newlines,num_words,num_unique_words,num_sentences,num_upper_sentences,num_lower_sentences,avg_word_size,num_small_words,freq_small_words,num_big_words,freq_big_words,num_huge_words,freq_huge_words,avg_sentence_chars,avg_sentence_words,std_sentence_chars,std_sentence_words,min_sentence_chars,min_sentence_words,max_sentence_chars,max_sentence_words
0,2377,False,1884,36,264,161,11,9,2,4.810606,111,0.420455,59,0.223485,1,0.003788,169.545455,24.090909,193.375924,19.681346,5,1,754,79
1,592,True,1311,47,195,60,9,9,0,5.169231,70,0.358974,59,0.302564,0,0.000000,144.333333,21.666667,103.887760,16.653328,8,1,300,48
2,1049,False,33,0,5,5,1,1,0,5.600000,2,0.400000,2,0.400000,0,0.000000,33.000000,5.000000,0.000000,0.000000,33,5,33,5
3,1087,True,1636,18,242,82,10,10,0,5.371901,94,0.388430,80,0.330579,0,0.000000,162.500000,24.200000,35.268258,6.968501,117,17,217,36
4,725,False,784,2,115,81,7,7,0,5.617391,37,0.321739,43,0.373913,0,0.000000,110.285714,16.428571,53.424752,7.798482,30,4,195,30
